# 正文
需要学习re和class类的定义


In [1]:
import requests 
from bs4 import BeautifulSoup
import re 
import collections
import math 
import time


In [2]:
class comment(object):
    """
    关于评论的对象
    参数：
    stars: 推荐程度
    content：评论的内容 
    time ：评论时间 
    
    """
    def __init__(self,stars,content,Time,vote):
        self.stars = stars
        self.content = content
        self.Time = Time
        self.vote = vote 
    
    def getContent(self):
        return content

In [3]:
class tv(object):
    """
    一个以电视剧作为对象的类  
    参数：
    Dict：指豆瓣爬虫返回的JSON对象，解析化后获得的dict
    属性： 
    title：电视剧题目 
    rank：电视剧评分 
    information：电视剧的主演等介绍 
    introduction：电视剧的剧情简介 
    comments：电视剧相关的评论的集合，集合中 是一系列 comment 对象 
    url：电视剧对应的豆瓣页面的链接 
    cover：电视剧的封面图链接 
    """
    def __init__(self,Dict):
        self.title = Dict['title']
        self.rank = Dict['rate']
        self.information = ''
        self.introduction = '' 
        self.comments = []
        self.url = Dict['url']
        self.cover = Dict['cover']
        
        
    def showinformation(self):
        #print(self.title)
        pass
    def getInformation(self,headers,downloadAllComments = True,Turns= 5):
        
        r = requests.get(self.url,headers=headers)
        soup = BeautifulSoup(r.text,'html.parser')
        self.information = soup.find('div',id='info').text
        self.introduction = soup.find('div',class_='related-info').text
        if downloadAllComments == True:    
            firstStep = soup.find('div',id='comments-section').find('span',class_='pl')
            href = firstStep.find('a')['href']
            
            num = int(re.search(r'[\d]+',firstStep.text).group(0))#获取总评论数、
            #避免评论太多爬取时间过长
            if Turns:
                RequestsTimes = Turns 
            else:
                RequestsTimes = math.ceil(num/20)
                
            for once in range(RequestsTimes):
                start = once*20 
                commentsUrl = href +'&start={}&limit=20&sort=new_score&percent_type='.format(start)
                response = requests.get(commentsUrl,headers=headers)
                soup1 = BeautifulSoup(response.text,'html.parser')
                time.sleep(0.1)
                try:
                    commentsList = soup1.find('div',class_= 'mod-bd').find_all('div',class_='comment-item')
                    for com in commentsList:
                        stars = com.find('span',class_=re.compile('rating'))['title']
                        Time = com.find('span',class_='comment-time')['title']
                        content=com.find('span',class_='short').text
                        vote = com.find('span',class_='votes').text
                        COM = comment(stars=stars,Time=Time,content=content,vote=vote)
                        self.comments.append(COM)
                except:
                    break 


            

        """
        commentsList =soup.find('div',id='hot-comments').find_all('div',class_='comment-item')
        for com in commentsList:            
            stars = com.find('span',class_=re.compile('rating'))['title']
            time = com.find('span',class_='comment-time')['title']
            content=com.find('span',class_='short').text
            COM = comment(stars=stars,time=time,content=content) 
            self.comments.append(COM)

        
        """


In [4]:

class doubanSpider(object):
    """
    豆瓣爬虫，是一个爬取豆瓣电视剧的内容的爬虫，其主要 爬取每一个电视剧的相关介绍及其评论（目前只有热门评论） 
    参数： 
    nums：想要爬取的电视剧总数 
    
    属性：
    head ： 爬虫的头部信息 
    url
    TVList：爬虫获取的所有电视剧的集合，其中的每一项都是tv对象
    
    """
    def __init__(self,nums):
        self.Totalnums = nums
        self.head = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36',
  'Cookie': 'bid="bSK74mb3o0U"; __gads=ID=81316fb8c6d49141:T=1581057714:S=ALNI_MY6rZqhAl57J0nESAyAyXvRgncNxA; douban-fav-remind=1; ll="108309"; gr_user_id=9d59bb85-5727-4d65-b377-18d1313eb1ca; _vwo_uuid_v2=DA4049E15DA0964D449EC0FD2852882E7|f98f0a5b2d5e82a5c914783eb1008f84; __utmz=30149280.1587041896.8.4.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); viewed="26022002"; ap_v=0,6.0; __utma=30149280.549375521.1581057714.1587041896.1587105876.9; __utmc=30149280; __utmb=30149280.10.10.1587105876; _pk_ses.100001.4cf6=*; __utma=223695111.1522244183.1587106246.1587106246.1587106246.1; __utmb=223695111.0.10.1587106246; __utmc=223695111; __utmz=223695111.1587106246.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __yadk_uid=5hqk227SXsAs0mKJ4P6VisbXsOP8syIv; _pk_id.100001.4cf6=76d2769dc72693ea.1587106246.1.1587106446.1587106246.'
}
        self.url = "https://movie.douban.com/j/search_subjects?type=tv&tag=热门&sort=recommend&page_limit=20&page_start={}"
        #self.dict = collections.OrderedDict()
        self.TVList = []
        
    def get_List(self):
        
        TotalTurn = math.ceil(self.Totalnums/20)
        
        index1 = TotalTurn*20 #总的需要下载的数目
        for startNum in range(TotalTurn):
            start = startNum*20
             
            url = self.url.format(start)
            r= requests.get(url,headers=self.head)
            List = r.json()['subjects']
            for i in List:
                TV = tv(i)
                TV.getInformation(self.head)
                self.TVList.append(TV)
                time.sleep(0.5)
                print('正在爬取：<{}>的相关信息 '.format(TV.title)) 
            index2 = start+20 # 记录已经下载了多少数据
            print('已经完成了{:.2f}%'.format(index2/index1))



In [5]:
%%time
t= doubanSpider(nums = 5)
t.get_List()

正在爬取：<我是余欢水>的相关信息 
正在爬取：<龙岭迷窟>的相关信息 
正在爬取：<十八年后的终极告白>的相关信息 
正在爬取：<口罩猎人>的相关信息 
正在爬取：<重生>的相关信息 
正在爬取：<清平乐>的相关信息 
正在爬取：<猎狐>的相关信息 
正在爬取：<国王：永远的君主>的相关信息 
正在爬取：<杀死伊芙 第三季>的相关信息 
正在爬取：<夫妻的世界>的相关信息 
正在爬取：<鬓边不是海棠红>的相关信息 
正在爬取：<民国奇探>的相关信息 
正在爬取：<机智医生生活>的相关信息 
正在爬取：<叹息桥>的相关信息 
正在爬取：<美国夫人>的相关信息 
正在爬取：<冰糖炖雪梨>的相关信息 
正在爬取：<陈情令>的相关信息 
正在爬取：<西部世界 第三季>的相关信息 
正在爬取：<那个男人的记忆法>的相关信息 
正在爬取：<请回答1988>的相关信息 
已经完成了1.00%
Wall time: 1min 5s


In [9]:
import  mysql.connector

In [30]:
conn = mysql.connector.connect(user='root', password='iamyourtf618',database='douban')
cursor = conn.cursor()

In [32]:
cursor.execute('DROP DATABASE douban')

DatabaseError: 1008 (HY000): Can't drop database 'douban'; database doesn't exist

In [55]:
t.TVList[0].rank

'7.4'